In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

In [2]:
# === Behavioural Model ===
class Behavioural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)
behavior_pi = Behavioural(input_dim=5, hidden_dim=64, output_dim=2)

# 2. Load the weights into it
behavior_pi.load_state_dict(torch.load("./Behavioural_model.pth"))


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/kevinkang/anaconda3/envs/Multiagent_Search/lib/python3.11/site-packages/ipykernel/

<All keys matched successfully>

In [4]:
# === Perturbation Model ===
class Policy(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)
pi = torch.load("./Policys/Perturbed_model_0.pth")

In [6]:
# KL between 2D Bernoulli probabilities for one batch
def bernoulli_kl(p, q, eps=1e-8):
    p = torch.clamp(p, eps, 1 - eps)
    q = torch.clamp(q, eps, 1 - eps)
    return (p * torch.log(p / q) + (1 - p) * torch.log((1 - p) / (1 - q))).sum(dim=-1)


In [7]:
# Compute KL between two MLP policies over a batch of states
def kl_between_policies(pi, pi_perturbed, states):
    with torch.no_grad():
        probs_pi = pi(states)
        probs_pert = pi_perturbed(states)
        kl_vals = bernoulli_kl(probs_pert, probs_pi)
        return kl_vals.mean()


In [ ]:
# Compute KL over multiple perturbed models
def kl_over_perturbed_set(pi, perturbed_pis, states):
    kl_results = []
    for i, perturbed_pi in enumerate(perturbed_pis):
        kl_val = kl_between_policies(pi, perturbed_pi, states)
        kl_results.append(kl_val.item())
        print(f"KL(pi || pi_perturbed_{i}) = {kl_val:.6f}")
    return kl_results


In [8]:
# Compute KL over multiple perturbed models
def kl_over_perturbed_set(pi, perturbed_pi, states):

    kl_val = kl_between_policies(pi, perturbed_pi, states)
    return kl_val

In [13]:
df = pd.read_csv("traj.csv")  # or use your DataFrame directly

states = torch.tensor(df.iloc[:40000, :5].values, dtype=torch.float32)
# states = torch.randn(100, 5)  # input_dim = 5

# Suppose this is your original policy
# pi = Behavioural(input_dim=5, hidden_dim=64, output_dim=2)

# # Create N perturbed copies
# perturbed_pis = [
#     get_multiplicatively_perturbed_copy(pi, omega=0.02)
#     for _ in range(10)
# ]

# Compute KL divergence for each
kl_scores = kl_over_perturbed_set(pi, behavior_pi, states)


In [14]:
kl_scores

tensor(0.0117)